## HF/LF/VLF analysis

See vsb_analyze-power-line-signals_parallel.py for HF/LF signal division
also vsb_signal_shift.py and vsb_spectrograms_ts_shift_parallel.py for time shift of the hf component for augmentation

In [1]:
import csv
import numpy as np
from scipy.interpolate import interp1d
from scipy import fftpack, signal
from scipy.signal import butter, filtfilt, hilbert
from scipy.signal import sosfilt, sosfreqz
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import pyarrow.parquet as pq
import pickle
import re, os
import random
from random import randint
from concurrent.futures import ProcessPoolExecutor
from collections import Collection
from pathlib import Path
from os import listdir
from os.path import isfile, join
import psutil
import time

/home/walle/miniconda3/envs/fastai-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  app.launch_new_instance()


In [2]:
pid = os.getpid()
py = psutil.Process(pid)

In [3]:
#import datashader as ds
#from datashader import transfer_functions as tf

In [4]:
period = 0.02
time_step = 0.02 / 800000.
time_vec = np.arange(0, 0.02, time_step)
f_sampling = 1 / time_step
print(f'Sampling Frequency = {f_sampling / 1e6} MHz')
# print (str(50* 800000 /1e6) + ' MHz')
samples = 800000
period = 0.02
sr = period / samples

Sampling Frequency = 40.0 MHz


### Good signal ts augmentation

too memory intensive to do so broke eack ts shift into 10 parts, here we recombine each

In [3]:
df_25_0=pq.read_table(f'../data/ML_Data/kaggle/VSB_Power_Line_Fault_Detection/bp_signals/shift_neg_sig_df_25_0.parquet').to_pandas()

In [4]:
df_25_0.tail()

,0,1,2,6,7,8,9,10,11,12,...,863,864,865,866,867,868,869,870,871,872
799995,-0.710341,1.038502,1.333920,1.641136,-0.211220,2.863419,-3.937174,-1.816853,-2.652810,-1.838458,...,-2.819275,-0.940728,-0.537838,0.753145,-5.672725,0.189435,-1.830618,0.654972,2.996547,2.870415
799996,-1.651654,-1.024932,-0.666129,-1.216273,-1.239267,1.956597,-2.892940,-0.912583,-1.411440,-1.936189,...,-2.702820,-0.940623,0.462208,0.854969,1.747772,2.080013,0.188900,-1.225242,-1.003389,-1.192668
799997,-1.651618,0.053726,-0.666177,-0.216391,-2.382757,0.897969,-1.259067,-0.967450,-1.411235,-1.955686,...,-3.196652,-0.996949,-0.537746,-0.207145,1.764630,2.386599,1.205376,-0.331908,-1.003325,-1.123119
799998,0.334480,1.027904,1.427139,1.729987,0.773245,3.863438,-2.016030,-0.830643,-1.425139,-0.869619,...,-2.905836,-0.874584,-0.537699,0.784072,-6.646045,0.206657,-1.830465,-0.348632,1.086445,2.842220
799999,-0.651548,0.059695,0.347076,-1.994963,-2.226634,-0.126996,-3.144819,0.090584,-0.443689,-2.053343,...,-3.196187,-1.940309,-0.569912,-0.216079,-2.303453,0.189647,-1.866304,-0.332154,-1.003197,-0.168745


In [5]:
df_25_0_values=df_25_0.values

In [6]:
df_25_0_values[:10]

array([[-6.77573284e-01,  5.96950089e-02,  3.37015447e-01, ...,
        -2.30895435e+00, -9.93101941e-01, -1.27339352e+00],
       [ 3.48522425e-01,  1.05969483e+00,  3.33630484e-01, ...,
         6.67599729e-01,  1.89708412e+00,  2.55819992e+00],
       [-2.54538468e+00, -1.94030536e+00, -1.73307439e+00, ...,
        -3.32523428e-01, -3.15568425e-03, -1.70079274e-01],
       ...,
       [-6.38240963e-01,  5.67088236e-02,  3.30008149e-01, ...,
         2.74686655e+00,  2.72755600e+00,  3.72760654e+00],
       [-6.51231058e-01,  6.44686943e-02,  3.39959303e-01, ...,
        -5.33326245e+00, -5.00262176e+00, -4.15703184e+00],
       [-2.91631540e+00, -1.02493467e+00, -1.53341431e+00, ...,
         1.66661436e+00,  2.99744219e+00,  2.87147342e+00]])

In [8]:
df_25_0_values.shape, type(df_25_0_values)

((800000, 825), numpy.ndarray)

In [5]:
path = ('../data/ML_Data/kaggle/VSB_Power_Line_Fault_Detection/bp_signals/')

In [6]:
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

In [7]:
onlyfiles[:5]

['test_vlf_hf_sig_1.parquet',
 'test_lf_sig_9.parquet',
 'test_vlf_hf_sig_6.parquet',
 'test_vlf_hf_sig_7.parquet',
 'test_hf_sig_8.parquet']

In [8]:
#check neg files processed
for n in [125, 25, 375]:
    for i in range(10):
        if (f'shift_neg_sig_df_{n}_{i}.parquet') not in onlyfiles:
            print(f'shift_neg_sig_df_{n}_{i}.parquet')

In [8]:
#check all pos files processed
for n in [125, 25, 375, 5, 625, 75, 875]:
    for i in range(10):
        if (f'shift_pos_sig_df_{n}_{i}.parquet') not in onlyfiles:
            print(f'shift_pos_sig_df_{n}_{i}.parquet')

In [28]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7,8]])
a, b

(array([[1, 2],
        [3, 4]]), array([[5, 6],
        [7, 8]]))

In [29]:
c=np.concatenate((a, b), axis=1)
c

array([[1, 2, 5, 6],
       [3, 4, 7, 8]])

In [30]:
df = pd.DataFrame(c)
df.head()

,0,1,2,3
0,1,2,5,6
1,3,4,7,8


In [9]:
#neg 5, 625, 75, 875 not generated, todo if have time
for n in [125, 25, 375]:
    df_l = []
    for i in range(10):
        start=time.time()
        df = pq.read_table(path+f'shift_neg_sig_df_{n}_{i}.parquet').to_pandas()
        vals = df.values
        df=None
        if i > 0:
            np_vals = np.concatenate((np_vals, vals), axis=1)
        else:
            np_vals=vals
        memoryUse = py.memory_info()[0] / 2. ** 30
        if memoryUse > 80.0:
            print('Too close to killing other process, exiting')
            break
        end = time.time()
        print(f'ts: {n}_{i} memory use: {memoryUse}, np_vals shape: {np_vals.shape}, elapsed: {end-start}')
    df = pd.DataFrame(np_vals)
    df.columns = df.columns.astype(str)
    memoryUse = py.memory_info()[0] / 2. ** 30
    print(f'df memory use: {memoryUse}')
    df.to_parquet(path+f'shift_neg_sig_df_{n}_good.parquet')

ts: 125_0 memory use: 5.081230163574219, np_vals shape: (800000, 825), elapsed: 57.165807247161865
ts: 125_1 memory use: 14.92453384399414, np_vals shape: (800000, 1650), elapsed: 66.51425051689148
ts: 125_2 memory use: 19.843059539794922, np_vals shape: (800000, 2475), elapsed: 70.8367350101471
ts: 125_3 memory use: 24.75998306274414, np_vals shape: (800000, 3300), elapsed: 70.53161859512329
ts: 125_4 memory use: 29.680404663085938, np_vals shape: (800000, 4125), elapsed: 71.41003441810608
ts: 125_5 memory use: 34.59574508666992, np_vals shape: (800000, 4950), elapsed: 70.23253011703491
ts: 125_6 memory use: 39.51633071899414, np_vals shape: (800000, 5775), elapsed: 72.87873435020447
ts: 125_7 memory use: 44.43178939819336, np_vals shape: (800000, 6600), elapsed: 77.06377863883972
ts: 125_8 memory use: 49.314876556396484, np_vals shape: (800000, 7422), elapsed: 79.86786675453186
ts: 125_9 memory use: 54.21613693237305, np_vals shape: (800000, 8244), elapsed: 81.33515334129333
df memor

In [9]:
#all generated for positive shift
for n in [125, 25, 375, 5, 625, 75, 875]:
    df_l = []
    for i in range(10):
        start=time.time()
        df = pq.read_table(path+f'shift_pos_sig_df_{n}_{i}.parquet').to_pandas()
        vals = df.values
        df=None
        if i > 0:
            np_vals = np.concatenate((np_vals, vals), axis=1)
        else:
            np_vals=vals
        memoryUse = py.memory_info()[0] / 2. ** 30
        if memoryUse > 80.0:
            print('Too close to killing other process, exiting')
            break
        end = time.time()
        print(f'ts: {n}_{i} memory use: {memoryUse}, np_vals shape: {np_vals.shape}, elapsed: {end-start}')
    df = pd.DataFrame(np_vals)
    df.columns = df.columns.astype(str)
    memoryUse = py.memory_info()[0] / 2. ** 30
    print(f'df memory use: {memoryUse}')
    df.to_parquet(path+f'shift_pos_sig_df_{n}_good.parquet')

ts: 125_0 memory use: 5.080818176269531, np_vals shape: (800000, 825), elapsed: 54.06538105010986
ts: 125_1 memory use: 14.923049926757812, np_vals shape: (800000, 1650), elapsed: 65.06323289871216
ts: 125_2 memory use: 19.842456817626953, np_vals shape: (800000, 2475), elapsed: 65.59692335128784
ts: 125_3 memory use: 24.75878143310547, np_vals shape: (800000, 3300), elapsed: 66.77323293685913
ts: 125_4 memory use: 29.678668975830078, np_vals shape: (800000, 4125), elapsed: 66.43538570404053
ts: 125_5 memory use: 34.595664978027344, np_vals shape: (800000, 4950), elapsed: 66.92087006568909
ts: 125_6 memory use: 39.51517105102539, np_vals shape: (800000, 5775), elapsed: 68.05065369606018
ts: 125_7 memory use: 44.434200286865234, np_vals shape: (800000, 6600), elapsed: 67.77844667434692
ts: 125_8 memory use: 49.315452575683594, np_vals shape: (800000, 7422), elapsed: 70.61985516548157
ts: 125_9 memory use: 54.2164306640625, np_vals shape: (800000, 8244), elapsed: 131.81215524673462
df me

### QC

In [ ]:
n=125
df_125=pq.read_table(f'../data/ML_Data/kaggle/VSB_Power_Line_Fault_Detection/bp_signals/shift_pos_sig_df_{n}_good.parquet').to_pandas()

In [ ]:
df_125.head()